In [45]:
!pip install faker
!pip install citipy
!jupyter nbextension enable --py --sys-prefix widgetsnbextension
!pip install gmaps
!jupyter nbextension enable --py --sys-prefix gmaps

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


Enabling notebook extension jupyter-gmaps/extension...
      - Validating: OK


In [83]:
from config import weather_api_key
import pandas as pd
import requests
import os
from faker import Faker
from citipy import citipy
import json

fake = Faker()
Faker.seed(0)

In [69]:
# Create 2,000 random lat longs and get the nearest city.
cities = []
for _ in range(2000):
    city = {}
    latlng = fake.latlng()
    city['city'] = citipy.nearest_city(float(latlng[0]), float(latlng[1])).city_name
    cities.append(city)
cities = pd.DataFrame(cities)
cities

,city
0,muros
1,rockport
2,torbay
3,palamos
4,saint-augustin
...,...
1995,kloulklubed
1996,attawapiskat
1997,rikitea
1998,port alfred


In [70]:
# Build the urls for the api call.
target_url = "https://api.openweathermap.org/data/2.5/weather?appid=" + weather_api_key
cities['search_query'] = cities['city'].replace(" ", "+", regex=True)
cities['url'] = target_url + "&q=" + cities['search_query']
pd.options.display.max_colwidth = 200
cities

,city,search_query,url
0,muros,muros,https://api.openweathermap.org/data/2.5/weather?appid=d576c2a3f77df113d247c8daeb1b9bc3&q=muros
1,rockport,rockport,https://api.openweathermap.org/data/2.5/weather?appid=d576c2a3f77df113d247c8daeb1b9bc3&q=rockport
2,torbay,torbay,https://api.openweathermap.org/data/2.5/weather?appid=d576c2a3f77df113d247c8daeb1b9bc3&q=torbay
3,palamos,palamos,https://api.openweathermap.org/data/2.5/weather?appid=d576c2a3f77df113d247c8daeb1b9bc3&q=palamos
4,saint-augustin,saint-augustin,https://api.openweathermap.org/data/2.5/weather?appid=d576c2a3f77df113d247c8daeb1b9bc3&q=saint-augustin
...,...,...,...
1995,kloulklubed,kloulklubed,https://api.openweathermap.org/data/2.5/weather?appid=d576c2a3f77df113d247c8daeb1b9bc3&q=kloulklubed
1996,attawapiskat,attawapiskat,https://api.openweathermap.org/data/2.5/weather?appid=d576c2a3f77df113d247c8daeb1b9bc3&q=attawapiskat
1997,rikitea,rikitea,https://api.openweathermap.org/data/2.5/weather?appid=d576c2a3f77df113d247c8daeb1b9bc3&q=rikitea
1998,port alfred,port+alfred,https://api.openweathermap.org/data/2.5/weather?appid=d576c2a3f77df113d247c8daeb1b9bc3&q=port+alfred


In [80]:
# Make api call and retrieve data
city_data = []
for city in cities.index:
    try:
        new_city = {}
        response = requests.get(cities['url'][city]).json()
        new_city['name'] = cities['city'][city]
        new_city['lat'] = response['coord']['lat']
        new_city['lng'] = response['coord']['lon']
        new_city['max_temp'] = response["main"]["temp_max"]
        new_city['percent_humidity'] = response['main']['humidity']
        new_city['percent_cloudiness'] = response['clouds']['all']
        new_city['wind_speed'] = response['wind']['speed']
        new_city['current_description'] = response['weather'][0]['description']
        city_data.append(new_city)
    except:
        print('City not found. Skipping...')
        pass
city_data = pd.DataFrame(city_data)
city_data

https://api.openweathermap.org/data/2.5/weather?appid=d576c2a3f77df113d247c8daeb1b9bc3&q=muros
{'coord': {'lon': -9.0603, 'lat': 42.7762}, 'weather': [{'id': 801, 'main': 'Clouds', 'description': 'few clouds', 'icon': '02n'}], 'base': 'stations', 'main': {'temp': 283.58, 'feels_like': 281.38, 'temp_min': 282.59, 'temp_max': 284.26, 'pressure': 1000, 'humidity': 81}, 'visibility': 10000, 'wind': {'speed': 2.24, 'deg': 285, 'gust': 6.26}, 'clouds': {'all': 21}, 'dt': 1612749855, 'sys': {'type': 3, 'id': 72671, 'country': 'ES', 'sunrise': 1612770164, 'sunset': 1612807093}, 'timezone': 3600, 'id': 3115824, 'name': 'Muros', 'cod': 200}
https://api.openweathermap.org/data/2.5/weather?appid=d576c2a3f77df113d247c8daeb1b9bc3&q=rockport
{'coord': {'lon': -97.0544, 'lat': 28.0206}, 'weather': [{'id': 800, 'main': 'Clear', 'description': 'clear sky', 'icon': '01n'}], 'base': 'stations', 'main': {'temp': 290.33, 'feels_like': 288.78, 'temp_min': 289.15, 'temp_max': 291.48, 'pressure': 1015, 'humidi

{'coord': {'lon': 115.3333, 'lat': -33.65}, 'weather': [{'id': 803, 'main': 'Clouds', 'description': 'broken clouds', 'icon': '04d'}], 'base': 'stations', 'main': {'temp': 292.04, 'feels_like': 293.44, 'temp_min': 292.04, 'temp_max': 292.04, 'pressure': 1000, 'humidity': 97}, 'visibility': 10000, 'wind': {'speed': 2.24, 'deg': 180, 'gust': 5.81}, 'clouds': {'all': 67}, 'dt': 1612749847, 'sys': {'type': 3, 'id': 2008303, 'country': 'AU', 'sunrise': 1612734450, 'sunset': 1612783096}, 'timezone': 28800, 'id': 2075265, 'name': 'Busselton', 'cod': 200}
https://api.openweathermap.org/data/2.5/weather?appid=d576c2a3f77df113d247c8daeb1b9bc3&q=fortuna
{'coord': {'lon': -124.1573, 'lat': 40.5982}, 'weather': [{'id': 800, 'main': 'Clear', 'description': 'clear sky', 'icon': '01n'}], 'base': 'stations', 'main': {'temp': 281.1, 'feels_like': 278.58, 'temp_min': 280.15, 'temp_max': 282.04, 'pressure': 1018, 'humidity': 93}, 'visibility': 10000, 'wind': {'speed': 2.57, 'deg': 270}, 'clouds': {'all': 

{'coord': {'lon': 132.75, 'lat': -5.6667}, 'weather': [{'id': 803, 'main': 'Clouds', 'description': 'broken clouds', 'icon': '04d'}], 'base': 'stations', 'main': {'temp': 302, 'feels_like': 302.31, 'temp_min': 302, 'temp_max': 302, 'pressure': 1007, 'humidity': 78, 'sea_level': 1007, 'grnd_level': 1007}, 'visibility': 10000, 'wind': {'speed': 8.4, 'deg': 293}, 'clouds': {'all': 84}, 'dt': 1612750033, 'sys': {'country': 'ID', 'sunrise': 1612732419, 'sunset': 1612776767}, 'timezone': 32400, 'id': 1623197, 'name': 'Tual', 'cod': 200}
https://api.openweathermap.org/data/2.5/weather?appid=d576c2a3f77df113d247c8daeb1b9bc3&q=burica
{'cod': '404', 'message': 'city not found'}
City not found. Skipping...
https://api.openweathermap.org/data/2.5/weather?appid=d576c2a3f77df113d247c8daeb1b9bc3&q=bluff
{'coord': {'lon': 168.3333, 'lat': -46.6}, 'weather': [{'id': 501, 'main': 'Rain', 'description': 'moderate rain', 'icon': '10d'}], 'base': 'stations', 'main': {'temp': 291.03, 'feels_like': 288.47, '

{'coord': {'lon': -6.5802, 'lat': 34.261}, 'weather': [{'id': 802, 'main': 'Clouds', 'description': 'scattered clouds', 'icon': '03n'}], 'base': 'stations', 'main': {'temp': 283.15, 'feels_like': 279.67, 'temp_min': 283.15, 'temp_max': 283.15, 'pressure': 1017, 'humidity': 93}, 'visibility': 6000, 'wind': {'speed': 4.63, 'deg': 210}, 'clouds': {'all': 28}, 'dt': 1612749970, 'sys': {'type': 1, 'id': 2402, 'country': 'MA', 'sunrise': 1612768689, 'sunset': 1612807377}, 'timezone': 3600, 'id': 2544571, 'name': 'Kenitra', 'cod': 200}
https://api.openweathermap.org/data/2.5/weather?appid=d576c2a3f77df113d247c8daeb1b9bc3&q=praia
{'coord': {'lon': -23.5087, 'lat': 14.9215}, 'weather': [{'id': 803, 'main': 'Clouds', 'description': 'broken clouds', 'icon': '04n'}], 'base': 'stations', 'main': {'temp': 293.15, 'feels_like': 287.59, 'temp_min': 293.15, 'temp_max': 293.15, 'pressure': 1017, 'humidity': 64}, 'visibility': 10000, 'wind': {'speed': 9.26, 'deg': 60}, 'clouds': {'all': 75}, 'dt': 161275

{'coord': {'lon': 100.3354, 'lat': 5.4112}, 'weather': [{'id': 801, 'main': 'Clouds', 'description': 'few clouds', 'icon': '02d'}], 'base': 'stations', 'main': {'temp': 299.89, 'feels_like': 304.18, 'temp_min': 298.71, 'temp_max': 301.15, 'pressure': 1011, 'humidity': 78}, 'visibility': 9000, 'wind': {'speed': 1.03, 'deg': 0}, 'clouds': {'all': 20}, 'dt': 1612749845, 'sys': {'type': 1, 'id': 9429, 'country': 'MY', 'sunrise': 1612740915, 'sunset': 1612783830}, 'timezone': 28800, 'id': 1735106, 'name': 'George Town', 'cod': 200}
https://api.openweathermap.org/data/2.5/weather?appid=d576c2a3f77df113d247c8daeb1b9bc3&q=vaitupu
{'cod': '404', 'message': 'city not found'}
City not found. Skipping...
https://api.openweathermap.org/data/2.5/weather?appid=d576c2a3f77df113d247c8daeb1b9bc3&q=okato
{'coord': {'lon': 173.8833, 'lat': -39.2}, 'weather': [{'id': 803, 'main': 'Clouds', 'description': 'broken clouds', 'icon': '04d'}], 'base': 'stations', 'main': {'temp': 293.99, 'feels_like': 291.73, 't

...

{'coord': {'lon': -114.3525, 'lat': 62.456}, 'weather': [{'id': 802, 'main': 'Clouds', 'description': 'scattered clouds', 'icon': '03n'}], 'base': 'stations', 'main': {'temp': 236.59, 'feels_like': 230.13, 'temp_min': 236.15, 'temp_max': 237.04, 'pressure': 1036, 'humidity': 67}, 'visibility': 10000, 'wind': {'speed': 3.6, 'deg': 320}, 'clouds': {'all': 40}, 'dt': 1612750022, 'sys': {'type': 1, 'id': 1007, 'country': 'CA', 'sunrise': 1612712880, 'sunset': 1612742112}, 'timezone': -25200, 'id': 6185377, 'name': 'Yellowknife', 'cod': 200}
https://api.openweathermap.org/data/2.5/weather?appid=d576c2a3f77df113d247c8daeb1b9bc3&q=mataura
{'coord': {'lon': 168.8643, 'lat': -46.1927}, 'weather': [{'id': 501, 'main': 'Rain', 'description': 'moderate rain', 'icon': '10d'}], 'base': 'stations', 'main': {'temp': 298.03, 'feels_like': 296.46, 'temp_min': 298.03, 'temp_max': 298.03, 'pressure': 1006, 'humidity': 33, 'sea_level': 1006, 'grnd_level': 1000}, 'visibility': 10000, 'wind': {'speed': 1.4, 

{'coord': {'lon': 167.7167, 'lat': -45.4167}, 'weather': [{'id': 502, 'main': 'Rain', 'description': 'heavy intensity rain', 'icon': '10d'}], 'base': 'stations', 'main': {'temp': 293.04, 'feels_like': 291.52, 'temp_min': 293.04, 'temp_max': 293.04, 'pressure': 1006, 'humidity': 57, 'sea_level': 1006, 'grnd_level': 982}, 'visibility': 10000, 'wind': {'speed': 2.68, 'deg': 345}, 'rain': {'1h': 13.315}, 'clouds': {'all': 100}, 'dt': 1612750040, 'sys': {'country': 'NZ', 'sunrise': 1612720488, 'sunset': 1612771913}, 'timezone': 46800, 'id': 2181625, 'name': 'Te Anau', 'cod': 200}
https://api.openweathermap.org/data/2.5/weather?appid=d576c2a3f77df113d247c8daeb1b9bc3&q=saskylakh
{'coord': {'lon': 114.0833, 'lat': 71.9167}, 'weather': [{'id': 802, 'main': 'Clouds', 'description': 'scattered clouds', 'icon': '03d'}], 'base': 'stations', 'main': {'temp': 235.09, 'feels_like': 227.47, 'temp_min': 235.09, 'temp_max': 235.09, 'pressure': 1028, 'humidity': 80, 'sea_level': 1028, 'grnd_level': 1027},

{'coord': {'lon': 17.0568, 'lat': 12.3807}, 'weather': [{'id': 804, 'main': 'Clouds', 'description': 'overcast clouds', 'icon': '04n'}], 'base': 'stations', 'main': {'temp': 291.44, 'feels_like': 286.79, 'temp_min': 291.44, 'temp_max': 291.44, 'pressure': 1011, 'humidity': 25, 'sea_level': 1011, 'grnd_level': 976}, 'visibility': 10000, 'wind': {'speed': 3.4, 'deg': 47}, 'clouds': {'all': 100}, 'dt': 1612750307, 'sys': {'country': 'TD', 'sunrise': 1612761355, 'sunset': 1612803366}, 'timezone': 3600, 'id': 2434985, 'name': 'Bokoro', 'cod': 200}
https://api.openweathermap.org/data/2.5/weather?appid=d576c2a3f77df113d247c8daeb1b9bc3&q=honningsvag
{'coord': {'lon': 25.9704, 'lat': 70.9821}, 'weather': [{'id': 600, 'main': 'Snow', 'description': 'light snow', 'icon': '13n'}], 'base': 'stations', 'main': {'temp': 269.94, 'feels_like': 259.82, 'temp_min': 269.94, 'temp_max': 269.94, 'pressure': 1023, 'humidity': 91, 'sea_level': 1023, 'grnd_level': 1017}, 'visibility': 3432, 'wind': {'speed': 1

{'coord': {'lon': 172.7902, 'lat': 3.0707}, 'weather': [{'id': 802, 'main': 'Clouds', 'description': 'scattered clouds', 'icon': '03d'}], 'base': 'stations', 'main': {'temp': 300.36, 'feels_like': 300.36, 'temp_min': 300.36, 'temp_max': 300.36, 'pressure': 1007, 'humidity': 79, 'sea_level': 1007, 'grnd_level': 1006}, 'visibility': 10000, 'wind': {'speed': 7.68, 'deg': 53}, 'clouds': {'all': 27}, 'dt': 1612750033, 'sys': {'country': 'KI', 'sunrise': 1612723375, 'sunset': 1612766591}, 'timezone': 43200, 'id': 2110227, 'name': 'Butaritari', 'cod': 200}
https://api.openweathermap.org/data/2.5/weather?appid=d576c2a3f77df113d247c8daeb1b9bc3&q=poum
{'coord': {'lon': 164.0167, 'lat': -20.2333}, 'weather': [{'id': 800, 'main': 'Clear', 'description': 'clear sky', 'icon': '01d'}], 'base': 'stations', 'main': {'temp': 300.46, 'feels_like': 301.94, 'temp_min': 300.46, 'temp_max': 300.46, 'pressure': 1005, 'humidity': 69, 'sea_level': 1005, 'grnd_level': 1005}, 'visibility': 10000, 'wind': {'speed'

{'cod': '404', 'message': 'city not found'}
City not found. Skipping...
https://api.openweathermap.org/data/2.5/weather?appid=d576c2a3f77df113d247c8daeb1b9bc3&q=ushuaia
{'coord': {'lon': -68.3, 'lat': -54.8}, 'weather': [{'id': 803, 'main': 'Clouds', 'description': 'broken clouds', 'icon': '04n'}], 'base': 'stations', 'main': {'temp': 283.15, 'feels_like': 280.99, 'temp_min': 283.15, 'temp_max': 283.15, 'pressure': 1011, 'humidity': 81}, 'visibility': 10000, 'wind': {'speed': 2.06, 'deg': 50}, 'clouds': {'all': 75}, 'dt': 1612750040, 'sys': {'type': 1, 'id': 8303, 'country': 'AR', 'sunrise': 1612689024, 'sunset': 1612743863}, 'timezone': -10800, 'id': 3833367, 'name': 'Ushuaia', 'cod': 200}
https://api.openweathermap.org/data/2.5/weather?appid=d576c2a3f77df113d247c8daeb1b9bc3&q=busselton
{'coord': {'lon': 115.3333, 'lat': -33.65}, 'weather': [{'id': 803, 'main': 'Clouds', 'description': 'broken clouds', 'icon': '04d'}], 'base': 'stations', 'main': {'temp': 292.04, 'feels_like': 293.44,

In [90]:
# Write the files to csv
filepath = os.path.join("WeatherPy_Database.csv")
city_data.to_csv(filepath, index=False)